<a href="https://colab.research.google.com/github/Reyqq/my_works/blob/main/lightgbm_library/lgbm_wrapper_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

# Определение функции для обучения модели
class LGBMWrapper:
    """LightGBM выполняет обучение на кросс-валидации и сохраняет лучшую модель.

    """
    # Определение конструктора
    def __init__(self, params=None):
        self.params = params if params is not None else {}
        self.model = None

    # Определение метода обучения
    def fit(self, X, y, test_size=0.2, random_state=42):
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

        self.model = lgb.train(self.params, train_data, valid_sets=[valid_data], early_stopping_rounds=10)

        y_pred = self.model.predict(X_valid, num_iteration=self.model.best_iteration)
        mse = mean_squared_error(y_valid, y_pred)
        print(f"Validation MSE: {mse:.4f}")

    # Определение метода предсказания
    def predict(self, X):
        if self.model is None:
            raise ValueError("Model is not trained yet!")
        return self.model.predict(X, num_iteration=self.model.best_iteration)

    # Определение метода оценки
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        mse = mean_squared_error(y, y_pred)
        return mse

    # Определение метода сохранения модели
    def save_model(self, filepath):
        self.model.save_model(filepath)

    # Определение метода загрузки модели
    def load_model(self, filepath):
        self.model = lgb.Booster(model_file=filepath)
